ctro = round.([ x_min + espacio[1]/2, y_min + espacio[2]/2,  z_min + espacio[3]/2 ], digits = 3)

dim = ncells * cutoff

half_dim = round(dim / 2, digits = 3)

sa = contacto.Voxel(ctro, half_dim, 0)

In [2]:
using Revise
using Chemfiles
using contacto
using StaticArrays

In [3]:
in_trj = Trajectory("/home/german/labo/contacto/aux/1mtn.pdb")
in_frm = read(in_trj)
in_top = Topology(in_frm)
coords = round.(positions(in_frm), digits = 3)
natoms = size(coords)[2]

#############

cutoff = round(5., digits = 3)
resolution = .1
t_xyz = transpose(coords)

srt_x_idx_t_xyz = sortperm(t_xyz[:, 1])
srt_x_t_xyz = t_xyz[srt_x_idx_t_xyz, 1]

x_min = srt_x_t_xyz[1]
x_max = srt_x_t_xyz[end]
y_min = minimum(t_xyz[:, 2])
y_max = maximum(t_xyz[:, 2])
z_min = minimum(t_xyz[:, 3])
z_max = maximum(t_xyz[:, 3])

espacio = [ x_max - x_min ; y_max - y_min ; z_max - z_min ]
ncells = convert(Int64, ceil(maximum(espacio) / cutoff))

#############

nlevels = convert(Int64, floor(log2(ncells)))
nnodes = sum([ 2^x for x in 1:(nlevels) ]) ; resto = (ncells - 2^(nlevels)) * 2 ; nnodes += resto
nodes = Array{Int64}(undef, nnodes+1);

nodes_anchos = Array{Array{Int64, 1}, 1}(undef, nlevels+1)
nodes_anchos[1] = [ncells]
nodes_ranges = Array{Array{Tuple{Int64,Int64}, 1}, 1}(undef, nlevels+1)
nodes_ranges[1] = [(0, ncells)]
nodes_bounds = Array{Array{Int64, 1}, 1}(undef, nlevels+1)
nodes_bounds[1] = [ncells]

let i = 1, j = 1; for i = 1:nlevels
    k = i + 1
    nodes_anchos[k] = []
    nodes_ranges[k] = []
    nodes_bounds[k] = []
    prev = 0
    for j = 1:2^(i-1)
        
        lh_node_ancho = convert(Int64, ceil(nodes_anchos[i][j] / 2))
        rh_node_ancho = nodes_anchos[i][j] - lh_node_ancho
        push!(nodes_anchos[k], lh_node_ancho)
        push!(nodes_anchos[k], rh_node_ancho)
        
        rango = nodes_ranges[i][j][2] - nodes_ranges[i][j][1]
        lh_node_rango = convert(Int64, ceil(rango / 2)) + prev
        rh_node_rango = rango + prev
        push!(nodes_ranges[k], (prev, lh_node_rango))
        push!(nodes_ranges[k], (lh_node_rango, rh_node_rango))
        prev = rh_node_rango
        
        push!(nodes_bounds[k], lh_node_rango)
       
    end
end
end

┌ Warning: `finalizer(o, f::Function)` is deprecated, use `finalizer(f, o)` instead.
│   caller = Type at Chemfiles.jl:35 [inlined]
└ @ Core /home/german/.julia/packages/Chemfiles/MzvVI/src/Chemfiles.jl:35
┌ Warning: `assert` is deprecated, use `@assert` instead.
│   caller = read(::Trajectory) at Trajectory.jl:36
└ @ Chemfiles /home/german/.julia/packages/Chemfiles/MzvVI/src/Trajectory.jl:36
┌ Warning: `finalizer(o, f::Function)` is deprecated, use `finalizer(f, o)` instead.
│   caller = Type at Chemfiles.jl:108 [inlined]
└ @ Core /home/german/.julia/packages/Chemfiles/MzvVI/src/Chemfiles.jl:108
┌ Warning: `assert` is deprecated, use `@assert` instead.
│   caller = ip:0x0
└ @ Core :-1
┌ Warning: `finalizer(o, f::Function)` is deprecated, use `finalizer(f, o)` instead.
│   caller = Type at Chemfiles.jl:51 [inlined]
└ @ Core /home/german/.julia/packages/Chemfiles/MzvVI/src/Chemfiles.jl:51
┌ Warning: `unsafe_wrap(T, pointer, dims, own)` is deprecated, use `unsafe_wrap(T, pointer, dims, o

In [4]:
nodes_anchos

4-element Array{Array{Int64,1},1}:
 [11]                    
 [6, 5]                  
 [3, 3, 3, 2]            
 [2, 1, 2, 1, 2, 1, 1, 1]

In [5]:
nodes_ranges

4-element Array{Array{Tuple{Int64,Int64},1},1}:
 [(0, 11)]                                                          
 [(0, 6), (6, 11)]                                                  
 [(0, 3), (3, 6), (6, 9), (9, 11)]                                  
 [(0, 2), (2, 3), (3, 5), (5, 6), (6, 8), (8, 9), (9, 10), (10, 11)]

In [6]:
nodes_bounds

4-element Array{Array{Int64,1},1}:
 [11]         
 [6]          
 [3, 9]       
 [2, 5, 8, 10]

In [8]:
nodes_indices_x = Array{Array{UnitRange{Int64}, 1}, 1}(undef, nlevels+1)
nodes_indices_x[1] = [1:natoms]
for i = 1:nlevels
    k = i + 1
    nodes_indices_x[k] = []
    for j = 1:2^(i-1)
        x_boundary = searchsortedlast(srt_x_t_xyz[nodes_indices_x[i][j]],
            (x_min + cutoff * nodes_bounds[k][j])) +
        nodes_indices_x[i][j].start - 1
        
        push!(nodes_indices_x[k],
            nodes_indices_x[i][j].start:x_boundary - 1)
        push!(nodes_indices_x[k],
            x_boundary:nodes_indices_x[i][j].stop)
    end
end

In [9]:
nodes_indices_x

4-element Array{Array{UnitRange{Int64},1},1}:
 [1:1989]                                                                   
 [1:744, 745:1989]                                                          
 [1:126, 127:744, 745:1719, 1720:1989]                                      
 [1:61, 62:126, 127:468, 469:744, 745:1453, 1454:1719, 1720:1924, 1925:1989]

In [88]:
idx_xy = findall(x -> x == tmp_srt_idx_t_xyz_y[lado_y][i],
            tmp_srt_idx_t_xyz_x[lado_x])

1-element Array{Int64,1}:
 27

In [ ]:
y_md_bound = searchsortedlast(srt_y_t_xyz[nodes_indices[i][j]],
    (y_min + cutoff * nodes_bounds[k][j]))
z_md_bound = searchsortedlast(srt_z_t_xyz[nodes_indices[i][j]],
    (z_min + cutoff * nodes_bounds[k][j]))

In [100]:
lado_y = 1
for i = 1:length(tmp_srt_idx_t_xyz_y[lado_y])
    lado_x = 1
    idx_xy = findall(x -> x == tmp_srt_idx_t_xyz_y[lado_y][i],
            tmp_srt_idx_t_xyz_x[lado_x])
    
    lado_z = 1
    if isempty(idx_xy)
    # está en -x, -y, 
        idx_zy = findall(z -> z == tmp_srt_idx_t_xyz_y[lado_y][i],
            tmp_srt_idx_t_xyz_z[lado_z])
        
        if isempty(idx_yz)
        # está en -x, -y, -z
            
        else
            
        end
    
        
    else
        
        if  srt_idx_t_xyz[x_lh_bound:x_rh_bound, 1][idx_xy] ==
        srt_idx_t_xyz[y_lh_bound:y_rh_bound, 2][i]
        
            lado_z = 1
                
            idx_zy = searchsortedlast(srt_idx_t_xyz[z_lh_bound:z_rh_bound, 3],
                srt_idx_t_xyz[y_lh_bound:y_rh_bound, 2][i])
            if idx_zy != 0
                if  srt_idx_t_xyz[z_lh_bound:z_rh_bound, 3][idx_zy] ==
                srt_idx_t_xyz[y_lh_bound:y_rh_bound, 2][i]
                
                    push!(octant_0, i)
                end
            end
        end
    end
end

4-element Array{Array{Int64,1},1}:
 [11]                    
 [6, 5]                  
 [3, 3, 3, 2]            
 [2, 1, 2, 1, 2, 1, 1, 1]

4-element Array{Array{Int64,1},1}:
 [11]                    
 [6, 5]                  
 [3, 3, 3, 2]            
 [2, 1, 2, 1, 2, 1, 1, 1]